In [6]:
!git clone https://github.com/am1tyadav/superhero

'git' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
with open('supername.txt','r') as f:
  data = f.read()

  data[:100] 
  

In [10]:
import glob
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
import tensorflow as tf
print(tf.__version__)

2.8.0


In [11]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

In [12]:
tokenizer.fit_on_texts(data)

In [19]:
char_to_index = tokenizer.word_index
index_to_char = dict((v,k) for k,v in char_to_index.items())

print(index_to_char)

{1: '\t', 2: 'a', 3: 'e', 4: 'r', 5: 'o', 6: 'n', 7: 'i', 8: ' ', 9: 't', 10: 's', 11: 'l', 12: 'm', 13: 'h', 14: 'd', 15: 'c', 16: 'u', 17: 'g', 18: 'k', 19: 'b', 20: 'p', 21: 'y', 22: 'w', 23: 'f', 24: 'v', 25: 'j', 26: 'z', 27: 'x', 28: 'q'}


In [20]:

names = data.splitlines()
names[:10]

['jumpa\t',
 'doctor fate\t',
 'starlight\t',
 'isildur\t',
 'lasher\t',
 'varvara\t',
 'the target\t',
 'axel\t',
 'battra\t',
 'changeling\t']

In [21]:
tokenizer.texts_to_sequences(names[0])

[[25], [16], [12], [20], [2], [1]]

In [22]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [23]:
name_to_seq(names[0])

[25, 16, 12, 20, 2, 1]

In [24]:
def seq_to_name(seq):
  return ''.join([index_to_char[i] for i in seq if i!= 0])

In [25]:
seq_to_name(name_to_seq(names[0]))

'jumpa\t'

In [27]:
sequences = []

for name in names:
  seq = name_to_seq(name)
  if len(seq) >= 2:
    sequences += [seq[:i] for i in range(2,len(seq) + 1)]

sequences[:10]

[[25, 16],
 [25, 16, 12],
 [25, 16, 12, 20],
 [25, 16, 12, 20, 2],
 [25, 16, 12, 20, 2, 1],
 [14, 5],
 [14, 5, 15],
 [14, 5, 15, 9],
 [14, 5, 15, 9, 5],
 [14, 5, 15, 9, 5, 4]]

In [28]:
max_len = max([len(x) for x in sequences])
print(max_len)

33


In [29]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences, padding = 'pre',
    maxlen = max_len
)
print(padded_sequences[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 25 16]


In [30]:
padded_sequences.shape

(88279, 33)

In [31]:
x,y = padded_sequences[:,:-1], padded_sequences[:, -1]
print(x.shape,y.shape)

(88279, 32) (88279,)


In [32]:
from sklearn.model_selection import train_test_split

x_train, x_test,y_train,y_test = train_test_split(x,y)

print(x_train.shape,y_train.shape)
print(x_test.shape, y_test.shape)

(66209, 32) (66209,)
(22070, 32) (22070,)


In [33]:
num_chars = len(char_to_index.keys()) + 1
print(num_chars)

29


In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Conv1D,MaxPool1D,LSTM
from tensorflow.keras.layers import Bidirectional, Dense 

model = Sequential([
        Embedding(num_chars,8 , input_length=max_len - 1) ,
        Conv1D(64,5,strides=1,activation ='tanh',padding = 'causal'),
        MaxPool1D(2),
        LSTM(32),
        Dense(num_chars,activation = 'softmax')            
                     ])

model.compile(
    loss = 'sparse_ctegorical_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 8)             232       
                                                                 
 conv1d (Conv1D)             (None, 32, 64)            2624      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 16, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 29)                957       
                                                                 
Total params: 16,229
Trainable params: 16,229
Non-trainable params: 0
____________________________________________________

In [40]:
h= model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=50, verbose=2,
    callbacks=[
               tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3)
    ]
)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\HANISH RAWAL\Python1\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\HANISH RAWAL\Python1\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HANISH RAWAL\Python1\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\HANISH RAWAL\Python1\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\HANISH RAWAL\Python1\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\HANISH RAWAL\Python1\lib\site-packages\keras\engine\compile_utils.py", line 184, in __call__
        self.build(y_pred)
    File "C:\